# Anonymisation
#### Ce script anonymise les champs du fichier DataSource.json d'une application PowerApps

In [2]:
import json
from collections import defaultdict
import re

def load_json_file(file_path):
    """Charge le fichier JSON"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"✅ Fichier chargé avec succès : {file_path}")
        return data
    except FileNotFoundError:
        print(f"❌ Erreur : Fichier non trouvé - {file_path}")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ Erreur : JSON invalide - {str(e)}")
        return None

def detect_structure(data):
    """Détecte automatiquement la structure du JSON"""
    if isinstance(data, list):
        print("🔍 Structure détectée : Tableau d'objets")
        return "array"
    elif isinstance(data, dict):
        print("🔍 Structure détectée : Objet unique")
        return "object"
    else:
        print(f"⚠️ Structure inattendue : {type(data)}")
        return "unknown"

def should_anonymize_type(type_value, name_value=None):
    """Détermine si un type doit être anonymisé"""
    if not type_value:
        return False
        
    if type_value == "ServiceInfo":
        return False
    elif type_value == "StaticDataSourceInfo":
        return False
    elif type_value == "NativeCDSDataSourceInfo":
        if name_value in ["Utilisateurs", "Divisions"]:
            return False
        return True
    elif type_value in ["ViewInfo", "OptionSetInfo"]:
        return True
    return False

def get_prefix_by_type(type_value):
    """Retourne le préfixe selon le type"""
    mapping = {
        "ViewInfo": "VI",
        "OptionSetInfo": "OSI",
        "NativeCDSDataSourceInfo": "TAB"
    }
    return mapping.get(type_value, "UNK")

def get_abbreviation(key):
    """Génère une abréviation à partir d'une clé"""
    # Mapping spécifique pour les clés courantes
    special_mapping = {
        "DatasetName": "DN",
        "DisplayName": "DN",
        "RelatedEntityName": "REN",
        "RelatedColumnInvariantName": "RCIN",
        "OptionSetReferenceEntityName": "OSREN",
        "OptionSetReferenceColumnName": "OSRCN",
        "OptionSetInfoNameMapping": "OSNM",
        "NativeCDSDataSourceInfoNameMapping": "NCDSDSINM",
        "EntitySetName": "ESN",
        "LogicalName": "LN"
    }
    
    if key in special_mapping:
        return special_mapping[key]
    
    # Extraction automatique des initiales
    abbrev = ""
    parts = re.findall(r'[A-Z][a-z]*', key)
    if parts:
        abbrev = ''.join([p[0] for p in parts])
    else:
        abbrev = key[:2].upper()
    return abbrev

def collect_all_values_for_anonymization(data):
    """Parcourt tout le JSON pour collecter toutes les valeurs à anonymiser"""
    global_value_mapping = {}
    type_counters = defaultdict(int)
    field_counters = defaultdict(lambda: defaultdict(int))
    processed_values = set()
    
    # Premier passage : collecter tous les Name principaux pour créer le mapping de référence
    name_to_anonymized = {}  # Mapping des noms originaux vers leurs versions anonymisées
    
    def collect_main_names(obj):
        if isinstance(obj, dict):
            if "Type" in obj and "Name" in obj:
                type_value = obj["Type"]
                name_value = obj["Name"]
                if should_anonymize_type(type_value, name_value):
                    if name_value not in name_to_anonymized:
                        prefix = get_prefix_by_type(type_value)
                        if prefix != "UNK":
                            type_counters[type_value] += 1
                            type_index = type_counters[type_value]
                            new_value = f"{prefix}_{type_index:02d}"
                            name_to_anonymized[name_value] = new_value
                            print(f"  📋 Mapping Name : '{name_value}' → '{new_value}' (Type: {type_value})")
            
            for key, value in obj.items():
                if isinstance(value, dict):
                    collect_main_names(value)
                elif isinstance(value, list):
                    for item in value:
                        if isinstance(item, dict):
                            collect_main_names(item)
    
    # Lancer le premier passage
    if isinstance(data, list):
        for item in data:
            collect_main_names(item)
    else:
        collect_main_names(data)
    
    def handle_value_anonymization(value, type_value, name_value, mapping, counters, field_key, parent_name=None):
        """Gère l'anonymisation d'une valeur avec nommage structuré"""
        if not should_anonymize_type(type_value, name_value):
            print(f"  🚫 Pas d'anonymisation pour : '{value}' (Type: {type_value}, Name: {name_value})")
            return False
        
        prefix = get_prefix_by_type(type_value)
        if prefix != "UNK":
            if value not in mapping:
                if parent_name is None:  # C'est un Name principal
                    # Utiliser le mapping pré-établi
                    if value in name_to_anonymized:
                        new_value = name_to_anonymized[value]
                        mapping[value] = new_value
                        print(f"  🔄 Anonymisation Name : '{value}' → '{new_value}' (Type: {type_value})")
                        return True
                    else:
                        counters[type_value] += 1
                        type_index = counters[type_value]
                        new_value = f"{prefix}_{type_index:02d}"
                        mapping[value] = new_value
                        print(f"  🔄 Anonymisation Name : '{value}' → '{new_value}' (Type: {type_value})")
                        return True
                else:  # C'est un champ dépendant
                    abbrev = get_abbreviation(field_key)
                    field_counters[parent_name][abbrev] += 1
                    field_index = field_counters[parent_name][abbrev]
                    new_value = f"{parent_name}_{abbrev}_{field_index:02d}"
                    mapping[value] = new_value
                    print(f"  🔄 Anonymisation champ : '{value}' → '{new_value}' (from: {parent_name})")
                    return True
            else:
                print(f"  🔁 Déjà anonymisé : '{value}' → '{mapping[value]}'")
                return True
        else:
            print(f"  ⚠️ Type inconnu : {type_value}")
            return False
        return False
    
    def recursive_collect(current_obj, path=[], parent_type=None, parent_name=None):
        if isinstance(current_obj, dict):
            # Récupérer le type et nom de l'objet courant
            current_type = current_obj.get("Type", parent_type)
            current_name = current_obj.get("Name", parent_name)
            
            # Vérifier si on doit anonymiser cet objet
            should_process = should_anonymize_type(current_type, current_name)
            
            # Si c'est un objet avec un Name, anonymiser ce Name
            if "Name" in current_obj and current_obj["Name"] not in processed_values:
                if should_process:
                    processed_values.add(current_obj["Name"])
                    success = handle_value_anonymization(current_obj["Name"], current_type, current_name, global_value_mapping, type_counters, "Name")
                    # Mettre à jour current_name avec la valeur anonymisée
                    if success and current_obj["Name"] in global_value_mapping:
                        current_name = global_value_mapping[current_obj["Name"]]
                else:
                    print(f"  🚫 Name non anonymisé : '{current_obj['Name']}' (Type: {current_type}, Name: {current_name})")
            
            # Parcourir toutes les paires clé/valeur
            for key, value in current_obj.items():
                new_path = path + [key]
                
                # Ne pas anonymiser la clé "Type"
                if key == "Type":
                    continue
                
                if isinstance(value, dict):
                    # Cas spécial : dictionnaire avec des valeurs string
                    if value and all(isinstance(k, str) and isinstance(v, str) for k, v in value.items()):
                        # Anonymiser les valeurs du dictionnaire
                        for dict_key, dict_value in value.items():
                            if isinstance(dict_value, str) and dict_value not in processed_values:
                                processed_values.add(dict_value)
                                if should_process:
                                    # Trouver le nom parent pour la structure
                                    parent_name_for_field = None
                                    if "Name" in current_obj and current_obj["Name"] in global_value_mapping:
                                        parent_name_for_field = global_value_mapping[current_obj["Name"]]
                                    elif current_name and current_name in global_value_mapping.values():
                                        parent_name_for_field = current_name
                                    elif current_name:
                                        parent_name_for_field = current_name
                                    
                                    handle_value_anonymization(dict_value, current_type, current_name, global_value_mapping, type_counters, key, parent_name_for_field)
                    
                    # Continuer la récursion
                    recursive_collect(value, new_path, current_type, current_name)
                    
                elif isinstance(value, list):
                    for i, item in enumerate(value):
                        if isinstance(item, dict):
                            recursive_collect(item, new_path + [i], current_type, current_name)
                        elif isinstance(item, str):
                            # Traiter les strings dans les listes
                            reference_keys = ["OptionSetReferenceEntityName", "OptionSetReferenceColumnName"]
                            if "Name" in str(key) or key in reference_keys:
                                if item not in processed_values:
                                    processed_values.add(item)
                                    if should_process:
                                        parent_name_for_field = None
                                        if "Name" in current_obj and current_obj["Name"] in global_value_mapping:
                                            parent_name_for_field = global_value_mapping[current_obj["Name"]]
                                        elif current_name and current_name in global_value_mapping.values():
                                            parent_name_for_field = current_name
                                        elif current_name:
                                            parent_name_for_field = current_name
                                        
                                        handle_value_anonymization(item, current_type, current_name, global_value_mapping, type_counters, key, parent_name_for_field)
                
                elif isinstance(value, str):
                    # Anonymiser si la clé est une clé de référence
                    reference_keys = [
                        "DatasetName", "DisplayName", "RelatedEntityName", 
                        "RelatedColumnInvariantName", "OptionSetReferenceEntityName", 
                        "OptionSetReferenceColumnName", "EntitySetName", "LogicalName", "PreferredName"
                    ]
                    if "Name" in str(key) or key in reference_keys:
                        if value not in processed_values:
                            processed_values.add(value)
                            if should_process:
                                # Trouver le nom parent
                                parent_name_for_field = None
                                if "Name" in current_obj and current_obj["Name"] in global_value_mapping:
                                    parent_name_for_field = global_value_mapping[current_obj["Name"]]
                                elif current_name and current_name in global_value_mapping.values():
                                    parent_name_for_field = current_name
                                elif current_name:
                                    parent_name_for_field = current_name
                                
                                handle_value_anonymization(value, current_type, current_name, global_value_mapping, type_counters, key, parent_name_for_field)
        
        elif isinstance(current_obj, list):
            for i, item in enumerate(current_obj):
                recursive_collect(item, path + [i], parent_type, parent_name)
    
    # Lancer la collecte
    if isinstance(data, list):
        for item in data:
            recursive_collect(item, [])
    else:
        recursive_collect(data, [])
    
    return global_value_mapping, name_to_anonymized

def anonymize_comprehensive(data, global_value_mapping, name_to_anonymized):
    """Anonymise complètement le JSON"""
    print("🔄 Application de l'anonymisation complète...")
    
    def recursive_anonymize(obj):
        if isinstance(obj, dict):
            new_obj = {}
            for key, value in obj.items():
                # NE PAS anonymiser la clé "Type"
                if key == "Type":
                    new_obj[key] = value
                    continue
                
                # Cas spécial : NativeCDSDataSourceInfoNameMapping - anonymiser les CLÉS
                if key == "NativeCDSDataSourceInfoNameMapping" and isinstance(value, dict):
                    new_dict = {}
                    for dict_key, dict_value in value.items():
                        # Anonymiser la clé avec le mapping Name -> anonymisé
                        new_key = name_to_anonymized.get(dict_key, dict_key)
                        new_value = global_value_mapping.get(dict_value, dict_value)
                        new_dict[new_key] = new_value
                        if dict_key in name_to_anonymized:
                            print(f"  🎯 Remplacement clé (NCDSDSINM): '{dict_key}' → '{name_to_anonymized[dict_key]}'")
                        if dict_value in global_value_mapping:
                            print(f"  🎯 Remplacement valeur (NCDSDSINM): '{dict_value}' → '{global_value_mapping[dict_value]}'")
                    new_obj[key] = new_dict
                # Cas spécial : autres dictionnaires avec valeurs string
                elif isinstance(value, dict):
                    if value and all(isinstance(k, str) and isinstance(v, str) for k, v in value.items()):
                        new_dict = {}
                        for dict_key, dict_value in value.items():
                            new_key = dict_key  # Ne pas anonymiser les clés normales
                            new_value = global_value_mapping.get(dict_value, dict_value)
                            new_dict[new_key] = new_value
                            if dict_value in global_value_mapping:
                                print(f"  🎯 Remplacement (dict value): '{dict_value}' → '{global_value_mapping[dict_value]}'")
                        new_obj[key] = new_dict
                    else:
                        new_obj[key] = recursive_anonymize(value)
                elif isinstance(value, list):
                    new_obj[key] = [recursive_anonymize(item) for item in value]
                elif isinstance(value, str):
                    if value in global_value_mapping:
                        new_obj[key] = global_value_mapping[value]
                        print(f"  🎯 Remplacement : {key} = '{value}' → '{global_value_mapping[value]}'")
                    else:
                        new_obj[key] = value
                else:
                    new_obj[key] = value
            return new_obj
        elif isinstance(obj, list):
            return [recursive_anonymize(item) for item in obj]
        else:
            return obj
    
    return recursive_anonymize(data)

def anonymize_json(data):
    """Fonction principale d'anonymisation"""
    print("🚀 Début de l'anonymisation...")
    
    # Détection de la structure
    structure_type = detect_structure(data)
    
    # Collecte de toutes les valeurs à anonymiser
    print("📋 Collecte exhaustive des valeurs à anonymiser...")
    global_value_mapping, name_to_anonymized = collect_all_values_for_anonymization(data)
    
    print(f"📊 Mapping final : {len(global_value_mapping)} valeurs à anonymiser")
    print(f"📋 Mapping Name : {len(name_to_anonymized)} correspondances Name → Anonymisé")
    
    if global_value_mapping:
        print("📄 Exemples de mappings :")
        # Afficher quelques exemples de chaque type
        type_examples = {}
        for original, anonymized in list(global_value_mapping.items())[:15]:
            prefix = anonymized.split('_')[0] if '_' in anonymized else anonymized
            if prefix not in type_examples:
                type_examples[prefix] = []
            if len(type_examples[prefix]) < 2:
                type_examples[prefix].append(f"    '{original}' → '{anonymized}'")
        
        for prefix, examples in type_examples.items():
            print(f"  {prefix} exemples:")
            for example in examples:
                print(f"    {example}")
        
        if len(global_value_mapping) > 15:
            print(f"    ... et {len(global_value_mapping) - 15} autres mappings")
    
    # Afficher le mapping Name -> Anonymisé
    if name_to_anonymized:
        print("📄 Mapping Name principal :")
        for i, (original, anonymized) in enumerate(list(name_to_anonymized.items())[:10]):
            print(f"    '{original}' → '{anonymized}'")
        if len(name_to_anonymized) > 10:
            print(f"    ... et {len(name_to_anonymized) - 10} autres")
    
    # Application de l'anonymisation
    print("🔄 Anonymisation complète du JSON...")
    anonymized_data = anonymize_comprehensive(data, global_value_mapping, name_to_anonymized)
    
    print("✅ Anonymisation terminée !")
    return anonymized_data

def save_json_file(data, output_path):
    """Sauvegarde les données dans un fichier JSON"""
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"💾 Fichier anonymisé sauvegardé : {output_path}")
        return True
    except Exception as e:
        print(f"❌ Erreur lors de la sauvegarde : {str(e)}")
        return False



def main():
    """Fonction principale"""
    input_file = "./data/DataSources.json"  # À modifier selon ton besoin
    output_file = "./data/anonymized_DataSource.json"
    
    print("📄 Script d'anonymisation de fichier JSON")
    print("=" * 50)
    
    # Chargement du fichier
    data = load_json_file(input_file)
    if data is None:
        return
    
    # Anonymisation
    anonymized_data = anonymize_json(data)
    
    # Sauvegarde
    if anonymized_data:
        save_json_file(anonymized_data, output_file)
    
    print("=" * 50)
    print("🏁 Processus terminé")

if __name__ == "__main__":
    main()

📄 Script d'anonymisation de fichier JSON
✅ Fichier chargé avec succès : ./data/DataSources.json
🚀 Début de l'anonymisation...
🔍 Structure détectée : Objet unique
📋 Collecte exhaustive des valeurs à anonymiser...
  📋 Mapping Name : 'cr446_especec' → 'OSI_01' (Type: OptionSetInfo)
  📋 Mapping Name : 'cr446_priorite' → 'OSI_02' (Type: OptionSetInfo)
  📋 Mapping Name : 'cr446_taille' → 'OSI_03' (Type: OptionSetInfo)
  📋 Mapping Name : 'cr446_tannage' → 'OSI_04' (Type: OptionSetInfo)
  📋 Mapping Name : 'smallapp_familleespece' → 'OSI_05' (Type: OptionSetInfo)
  📋 Mapping Name : 'smallapp_foulon' → 'OSI_06' (Type: OptionSetInfo)
  📋 Mapping Name : 'cr446_cuve_statecode' → 'OSI_07' (Type: OptionSetInfo)
  📋 Mapping Name : 'cr446_cuve_statuscode' → 'OSI_08' (Type: OptionSetInfo)
  📋 Mapping Name : 'smallapp_cr446_programme_smallapp_tri' → 'OSI_09' (Type: OptionSetInfo)
  📋 Mapping Name : 'cr446_decoupe' → 'OSI_10' (Type: OptionSetInfo)
  📋 Mapping Name : 'cr446_operations' → 'OSI_11' (Type: Op